# Guest Lecture COMP7230
## *Using Python packages for Linked Data & spatial data*
#### by Nicholas Car, SURROUND Australia Pty Ltd

This Notebook is the resource used to deliver a guest leacture for the [Austalian National University](https://www.anu.edu.au)'s course [COMP7230](https://programsandcourses.anu.edu.au/2020/course/COMP7230): *Introduction to Programming for Data Scientists*

## About the lecturer
**Nicholas Car**:
* PhD in informatics for irrigation
* A former CSIRO informatics researcher
    * worked on integrating environmental data across government / industry
    * developed data standards
* Has worked in operation IT in government
* Now in a private IT consulting company, [SURROUND Australia Pty Ltd](https://surroundaustralia.com) supplying Data Science solutions

Relevant current work:

* building data processing systems for government & industry
* mainly using Python
    * due to its large number of web and data science packages
* maintains the [RDFlib](https://rdflib.net) Python toolkit
    * for processing [RDF](https://en.wikipedia.org/wiki/Resource_Description_Framework)
* co-chairs the [Australian Government Linked Data Working Group](https://www.linked.data.gov.au) with Armin Haller
    * plans for multi-agency data integration
* still developing data standards


## 0. Lecture Outline
1. Notes about this training material
2. Accessing RDF data
3. Parsing RDF data
4. Data 'mash up'


## 1. Notes about this training material

* This is an Jupyter Notebook - interactive Python scripting
* You will cover Jupyter Notebooks more, later in this course
* Access this material online at:
    * GitHub: <https://github.com/nicholascar/comp7230-training>

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/nicholascar/comp7230-training/?filepath=lecture.ipynb)

A few words about RDF:
* Nick will talk through some web pages:
    * [Semantic Web](https://www.w3.org/standards/semanticweb/)
    * [RDF](https://en.wikipedia.org/wiki/Resource_Description_Framework)
    * [RDFlib](https://rdflib.net) Python toolkit
    * [RDFlib training Notebooks are available](https://github.com/surroundaustralia/rdflib-training)

The LocI project:
* The Location Index project: <http://loci.cat>

## 2. Accessing RDF data

* Here we use an online structured dataset, the Geocoded National Address File for Australia
    * Dataset Persistent Identifier: <https://linked.data.gov.au/dataset/gnaf>
    * The above link redirects to the API at <https://gnafld.net>
* GNAF-LD Data is presented according to *Linked Data* principles
    * online
    * in HTML & machine readable form, RDF
    * RDF is a Knowledge Graph: a graph containing data + model
    * each resource is available via a URI
        * e.g. <https://linked.data.gov.au/dataset/gnaf/address/GAACT714845933>

![GAACT714845933](img/GAACT714845933.png)


2.1. Get the Address GAACT714845933 using the *requests* package

In [167]:
import requests  # NOTE: you must have installed requests first, it's not a standard package
r = requests.get(
    "https://linked.data.gov.au/dataset/gnaf/address/GAACT714845933"
)
print(r.text)

<!DOCTYPE html>
<html>
<head lang="en">
    <meta charset="UTF-8">
    <title>Address API</title>
    <link rel="stylesheet" href="/static/css/psma_theme.css" />
</head>
<body>
    <div id="widther">
        <div id="header">
            <div style="float:left;">
                <a href="https://www.psma.com.au/">PSMA Australia Ltd.</a>
            </div>
            <div style="float:right;">
                <a href="/">Home</a>
                <a href="/?_view=reg">Registers</a>
                <a href="/sparql">SPARQL endpoint</a>
                <a href="http://linked.data.gov.au/def/gnaf">GNAF ontology</a>
                <a href="http://linked.data.gov.au/def/gnaf/code/">GNAF codes</a>
                <a href="/about">About</a>
            </div>
            <div style="clear:both;"></div>
        </div>
        <div id="container-content">
            
    <h1>Address GAACT714845933</h1>
    <script type="application/ld+json">
        {"@type": "Place", "name": "Geocoded Address

2.2 Get machine-readable data, RDF triples
Use HTTP Content Negotiation
Same URI, different *format* of data

In [168]:
r = requests.get(
    "https://linked.data.gov.au/dataset/gnaf/address/GAACT714845933",
    headers={"Accept": "application/n-triples"}
)
print(r.text)

<http://linked.data.gov.au/dataset/gnaf/address/GAACT714845933> <http://www.opengis.net/ont/geosparql#hasGeometry> _:N82500b1deacf4c8b96aad333f9f2cc73 .
_:N82500b1deacf4c8b96aad333f9f2cc73 <http://purl.org/dc/terms/type> <http://gnafld.net/def/gnaf/code/GeocodeTypes#FrontageCentreSetback> .
<http://linked.data.gov.au/dataset/gnaf/address/GAACT714845933> <http://linked.data.gov.au/def/gnaf#hasLocality> <http://linked.data.gov.au/dataset/gnaf/locality/ACT570> .
<http://linked.data.gov.au/dataset/gnaf/address/GAACT714845933> <http://linked.data.gov.au/def/gnaf#hasState> <http://www.geonames.org/2177478> .
_:Nf091db34f7a74593a58ca1d82eb42f9e <http://www.w3.org/ns/prov#value> "6"^^<http://www.w3.org/2001/XMLSchema#integer> .
<http://linked.data.gov.au/dataset/gnaf/address/GAACT714845933> <http://linked.data.gov.au/def/gnaf#hasPostcode> "2615"^^<http://www.w3.org/2001/XMLSchema#integer> .
<http://gnafld.net/def/gnaf/GnafConfidence_2> <http://www.w3.org/2000/01/rdf-schema#label> "Confidence l

2.3 Get machine-readable data, Turtle
Easier to read

In [169]:
r = requests.get(
    "https://linked.data.gov.au/dataset/gnaf/address/GAACT714845933",
    headers={"Accept": "text/turtle"}
)
print(r.text)

@prefix dct: <http://purl.org/dc/terms/> .
@prefix geo: <http://www.opengis.net/ont/geosparql#> .
@prefix gnaf: <http://linked.data.gov.au/def/gnaf#> .
@prefix loci: <http://linked.data.gov.au/def/loci#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sf: <http://www.opengis.net/ont/sf#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://linked.data.gov.au/dataset/gnaf/address/GAACT714845933> a gnaf:Address ;
    rdfs:label "Address GAACT714845933 of Unknown type"^^xsd:string ;
    gnaf:hasAddressSite <http://linked.data.gov.au/dataset/gnaf/addressSite/710446419> ;
    gnaf:hasDateCreated "2004-04-29"^^xsd:date ;
    gnaf:hasDateLastModified "2018-02-01"^^xsd:date ;
    gnaf:hasGnafConfidence <http://gnafld.net/def/gnaf/GnafConfidence_2> ;
    gnaf:hasLocality <http://linked.data.gov.au/dataset/gnaf/

## 3. Parsing RDF data

Import the RDFlib library for manipulating RDF data
Add some namespaces to shorten URIs

In [170]:
import rdflib
from rdflib.namespace import RDF, RDFS
GNAF = rdflib.Namespace("http://linked.data.gov.au/def/gnaf#")
ADDR = rdflib.Namespace("http://linked.data.gov.au/dataset/gnaf/address/")
GEO = rdflib.Namespace("http://www.opengis.net/ont/geosparql#")
print(GEO)

http://www.opengis.net/ont/geosparql#


Create a graph and add the namespaces to it

In [171]:
g = rdflib.Graph()
g.bind("gnaf", GNAF)
g.bind("addr", ADDR)
g.bind("geo", GEO)
print(g.serialize(format="text/turtle").decode())

Parse in the machine-readable data from the GNAF-LD
Print the length of the graph to check

In [172]:
r = requests.get(
    "https://linked.data.gov.au/dataset/gnaf/address/GAACT714845933",
    headers={"Accept": "text/turtle"}
)
g.parse(data=r.text, format="text/turtle")

<Graph identifier=N2893fbb14a284c1082727f197a8e9ec3 (<class 'rdflib.graph.Graph'>)>

Print graph length (no. of triples)

In [173]:
print(len(g))

28


Print graph content, in Turtle

In [174]:
print(g.serialize(format="text/turtle").decode())

@prefix addr: <http://linked.data.gov.au/dataset/gnaf/address/> .
@prefix dct: <http://purl.org/dc/terms/> .
@prefix geo: <http://www.opengis.net/ont/geosparql#> .
@prefix gnaf: <http://linked.data.gov.au/def/gnaf#> .
@prefix loci: <http://linked.data.gov.au/def/loci#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sf: <http://www.opengis.net/ont/sf#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

addr:GAACT714845933 a gnaf:Address ;
    rdfs:label "Address GAACT714845933 of Unknown type"^^xsd:string ;
    gnaf:hasAddressSite <http://linked.data.gov.au/dataset/gnaf/addressSite/710446419> ;
    gnaf:hasDateCreated "2004-04-29"^^xsd:date ;
    gnaf:hasDateLastModified "2018-02-01"^^xsd:date ;
    gnaf:hasGnafConfidence <http://gnafld.net/def/gnaf/GnafConfidence_2> ;
    gnaf:hasLocality <http://linked.data.gov.au/dataset/gnaf/locality/ACT570> ;
    gnaf:hasNumber [ a gnaf:Number ;
            gnaf:gnafType <http://lin

### 3.1 Getting multi-address data:
3.1.1. Retrieve an index of 10 addresses, in RDF
3.1.2. For each address in the index, get each Address' data
* use paging URI: <https://linked.data.gov.au/dataset/gnaf/address/?page=1>
3.1.3. Get only the street address and map coordinates

#### 3.1.1. Retrieve index

In [175]:
# clear the graph
g = rdflib.Graph()

r = requests.get(
    "https://linked.data.gov.au/dataset/gnaf/address/?page=1",
    headers={"Accept": "text/turtle"}
)
g.parse(data=r.text, format="text/turtle")
print(len(g))

70


#### 3.1.2. Parse in each address' data

In [176]:
for s, p, o in g.triples((None, RDF.type, GNAF.Address)):
    print(s.split("/")[-1])
    r = requests.get(
        str(s),
        headers={"Accept": "text/turtle"}
    )
    g.parse(data=r.text, format="turtle")
    print(len(g))

GAACT714845953
97
GAACT714845941
122
GAACT714845947
147
GAACT714845954
172
GAACT714845949
197
GAACT714845935
222
GAACT714845933
247
GAACT714845945
272
GAACT714845955
297
GAACT714845936
322
GAACT714845950
347
GAACT714845943
372
GAACT714845944
397
GAACT714845939
422
GAACT714845946
447
GAACT714845952
472
GAACT714845942
497
GAACT714845938
522
GAACT714845951
547
GAACT714845934
572


#### 3.1.3. Extract (& print) street address text & coordinates
(CSV)

In [177]:
addresses_tsv = "GNAF ID\tAddress\tCoordinates\n"
for s, p, o in g.triples((None, RDF.type, GNAF.Address)):
    for s2, p2, o2 in g.triples((s, RDFS.comment, None)):
        txt = str(o2)
    for s2, p2, o2 in g.triples((s, GEO.hasGeometry, None)):
        for s3, p3, o3 in g.triples((o2, GEO.asWKT, None)):
            coords = str(o3).replace("<http://www.opengis.net/def/crs/EPSG/0/4283> ", "")

    addresses_tsv += "{}\t{}\t{}\n".format(str(s).split("/")[-1], txt, coords)

print(addresses_tsv)

GNAF ID	Address	Coordinates
GAACT714845953	5 Jamieson Crescent, Kambah, ACT 2902	POINT(149.06864966 -35.37733591)
GAACT714845941	7 Mcdowall Place, Kambah, ACT 2902	POINT(149.06860919 -35.37833726)
GAACT714845947	20 Jauncey Court, Charnwood, ACT 2615	POINT(149.03705032 -35.19796828)
GAACT714845954	5 Baylis Place, Charnwood, ACT 2615	POINT(149.03048051 -35.20185603)
GAACT714845949	13 Mcdowall Place, Kambah, ACT 2902	POINT(149.06908395 -35.37882495)
GAACT714845935	26 Jauncey Court, Charnwood, ACT 2615	POINT(149.03640841 -35.19777173)
GAACT714845933	6 Packham Place, Charnwood, ACT 2615	POINT(149.03865604 -35.20113263)
GAACT714845945	9 Baylis Place, Charnwood, ACT 2615	POINT(149.03047333 -35.20156767)
GAACT714845955	3 Baylis Place, Charnwood, ACT 2615	POINT(149.03046282 -35.20202762)
GAACT714845936	17 Geeves Court, Charnwood, ACT 2615	POINT(149.03687042 -35.20395740)
GAACT714845950	7 Baylis Place, Charnwood, ACT 2615	POINT(149.03049843 -35.20169346)
GAACT714845943	22 Jauncey Court, Charnwoo

#### 3.1.4. Convert CSV data to PANDAS DataFrame
(CSV)

In [178]:
import pandas
from io import StringIO
s = StringIO(addresses_tsv)
df1 = pandas.read_csv(s, sep="\t")
print(df1)


           GNAF ID                                Address  \
0   GAACT714845953  5 Jamieson Crescent, Kambah, ACT 2902   
1   GAACT714845941     7 Mcdowall Place, Kambah, ACT 2902   
2   GAACT714845947  20 Jauncey Court, Charnwood, ACT 2615   
3   GAACT714845954    5 Baylis Place, Charnwood, ACT 2615   
4   GAACT714845949    13 Mcdowall Place, Kambah, ACT 2902   
5   GAACT714845935  26 Jauncey Court, Charnwood, ACT 2615   
6   GAACT714845933   6 Packham Place, Charnwood, ACT 2615   
7   GAACT714845945    9 Baylis Place, Charnwood, ACT 2615   
8   GAACT714845955    3 Baylis Place, Charnwood, ACT 2615   
9   GAACT714845936   17 Geeves Court, Charnwood, ACT 2615   
10  GAACT714845950    7 Baylis Place, Charnwood, ACT 2615   
11  GAACT714845943  22 Jauncey Court, Charnwood, ACT 2615   
12  GAACT714845944     9 Mcdowall Place, Kambah, ACT 2902   
13  GAACT714845939  24 Jauncey Court, Charnwood, ACT 2615   
14  GAACT714845946    11 Mcdowall Place, Kambah, ACT 2902   
15  GAACT714845952  18 J

#### 3.1.5. SPARQL querying RDF data
A graph query, similar to a database SQL query, can traverse the graph and retrieve the same details as the multiple
loops and Python code above in 3.1.3.

In [179]:
q = """
SELECT ?id ?addr ?coords
WHERE {
    ?uri a gnaf:Address ;
         rdfs:comment ?addr .

    ?uri geo:hasGeometry/geo:asWKT ?coords_dirty .

    BIND (STRAFTER(STR(?uri), "address/") AS ?id)
    BIND (STRAFTER(STR(?coords_dirty), "4283> ") AS ?coords)
}
ORDER BY ?id
"""
for r in g.query(q):
    print("{}, {}, {}".format(r["id"], r["addr"], r["coords"]))

GAACT714845933, 6 Packham Place, Charnwood, ACT 2615, POINT(149.03865604 -35.20113263)
GAACT714845934, 3 Bunker Place, Charnwood, ACT 2615, POINT(149.04011870 -35.19989093)
GAACT714845935, 26 Jauncey Court, Charnwood, ACT 2615, POINT(149.03640841 -35.19777173)
GAACT714845936, 17 Geeves Court, Charnwood, ACT 2615, POINT(149.03687042 -35.20395740)
GAACT714845938, 5 Mcdowall Place, Kambah, ACT 2902, POINT(149.06851657 -35.37815855)
GAACT714845939, 24 Jauncey Court, Charnwood, ACT 2615, POINT(149.03661902 -35.19784933)
GAACT714845941, 7 Mcdowall Place, Kambah, ACT 2902, POINT(149.06860919 -35.37833726)
GAACT714845942, 5 Bunker Place, Charnwood, ACT 2615, POINT(149.04029706 -35.19999611)
GAACT714845943, 22 Jauncey Court, Charnwood, ACT 2615, POINT(149.03688520 -35.19795303)
GAACT714845944, 9 Mcdowall Place, Kambah, ACT 2902, POINT(149.06872290 -35.37847955)
GAACT714845945, 9 Baylis Place, Charnwood, ACT 2615, POINT(149.03047333 -35.20156767)
GAACT714845946, 11 Mcdowall Place, Kambah, ACT 29

## 4. Data 'mash up'
Add some fake data to the GNAF data - people count per address

In [180]:
import pandas
df2 = pandas.read_csv('fake_data.csv')
print(df2)

           GNAF ID   Persons
0   GAACT714845944         3
1   GAACT714845934         5
2   GAACT714845943        10
3   GAACT714845949         1
4   GAACT714845955         2
5   GAACT714845935         1
6   GAACT714845947         4
7   GAACT714845950         3
8   GAACT714845933         4
9   GAACT714845953         2
10  GAACT714845945         3
11  GAACT714845946         3
12  GAACT714845939         4
13  GAACT714845941         2
14  GAACT714845942         1
15  GAACT714845954         0
16  GAACT714845952         5
17  GAACT714845938         3
18  GAACT714845936         4
19  GAACT714845951         3


Merge DataFrames

In [181]:
df3 = pandas.merge(df1, df2)
print(df3.head())

          GNAF ID                                Address  \
0  GAACT714845953  5 Jamieson Crescent, Kambah, ACT 2902   
1  GAACT714845941     7 Mcdowall Place, Kambah, ACT 2902   
2  GAACT714845947  20 Jauncey Court, Charnwood, ACT 2615   
3  GAACT714845954    5 Baylis Place, Charnwood, ACT 2615   
4  GAACT714845949    13 Mcdowall Place, Kambah, ACT 2902   

                        Coordinates   Persons  
0  POINT(149.06864966 -35.37733591)         2  
1  POINT(149.06860919 -35.37833726)         2  
2  POINT(149.03705032 -35.19796828)         4  
3  POINT(149.03048051 -35.20185603)         0  
4  POINT(149.06908395 -35.37882495)         1  


## 5. Spatial Data Display

#### 5.1 Import shapely tools

In [182]:
import shapely.wkt
from shapely.geometry import MultiPoint
import json

#### 5.2 Convert WKT to GeoJSON

In [183]:
points_list = []

for s, p, o in g.triples((None, RDF.type, GNAF.Address)):
    for s2, p2, o2 in g.triples((s, GEO.hasGeometry, None)):
        for s3, p3, o3 in g.triples((o2, GEO.asWKT, None)):
            g1 = shapely.wkt.loads(str(o3).replace("<http://www.opengis.net/def/crs/EPSG/0/4283> ", ""))
            points_list.append(g1)

mp = MultiPoint(points=points_list)

geojson = shapely.geometry.mapping(mp)
print(json.dumps(geojson, indent=4))

{
    "type": "MultiPoint",
    "coordinates": [
        [
            149.06864966,
            -35.37733591
        ],
        [
            149.06860919,
            -35.37833726
        ],
        [
            149.03705032,
            -35.19796828
        ],
        [
            149.03048051,
            -35.20185603
        ],
        [
            149.06908395,
            -35.37882495
        ],
        [
            149.03640841,
            -35.19777173
        ],
        [
            149.03865604,
            -35.20113263
        ],
        [
            149.03047333,
            -35.20156767
        ],
        [
            149.03046282,
            -35.20202762
        ],
        [
            149.03687042,
            -35.2039574
        ],
        [
            149.03049843,
            -35.20169346
        ],
        [
            149.0368852,
            -35.19795303
        ],
        [
            149.0687229,
            -35.37847955
        ],
        [
        

*Show results in QGIS*

## License
All the content in this repository is licensed under the [CC BY 4.0 license](https://creativecommons.org/licenses/by/4.0/). Basically, you can:

* copy and redistribute the material in any medium or format
* remix, transform, and build upon the material for any purpose, even commercially

You just need to:

* give appropriate credit, provide a link to the license, and indicate if changes were made
* not apply legal terms or technological measures that legally restrict others from doing anything the license permits

## Contact Information
**Dr Nicholas J. Car**<br />
*Data Systems Architect*<br />
[SURROUND Australia Pty Ltd](https://surroundaustralia.com)<br />
<nicholas.car@surroundaustralia.com><br />
GitHub: [nicholascar](https://github.com/nicholascar)<br />
ORCID: <https://orcid.org/0000-0002-8742-7730><br />